# Create NetApp Snapshot within Jupyter Notebook

This playbook demonstrates how to trigger the creation of a snapshot of a NetApp volume from within a Jupyter Notebook

## Install netapp_ontap module

In [1]:
%pip install --user netapp_ontap

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Import needed functions/classes

In [2]:
from netapp_ontap import config as netappConfig
from netapp_ontap.host_connection import HostConnection as NetAppHostConnection
from netapp_ontap.resources import Volume, Snapshot
from datetime import datetime
import json

## Configure connection to ONTAP cluster/instance

In [3]:
## Enter connection details for your ONTAP cluster/instance
ontapClusterMgmtHostname = '10.61.188.40'
ontapClusterAdminUsername = 'admin'
ontapClusterAdminPassword = 'NetApp!23'
verifySSLCert = False
##

netappConfig.CONNECTION = NetAppHostConnection(
    host = ontapClusterMgmtHostname,
    username = ontapClusterAdminUsername,
    password = ontapClusterAdminPassword,
    verify = verifySSLCert
)

## Convert pv name to ONTAP volume name

In [4]:
## Enter the name of pv for which you are creating a snapshot
##   Note: To get the name of the pv, you can run `kubectl -n <namespace> get pvc`.
##         The name of the pv that corresponds to a given pvc can be found in the 'VOLUME' 
##         column.
pvName = 'pvc-db963a53-abf2-4ffa-9c07-8815ce78d506'
##

# The following will not work if you specified a custom storagePrefix when creating your
#   Trident backend. If you specified a custom storagePrefix, you will need to update this
#   code to match your prefix.
volumeName = 'trident_%s' % pvName.replace("-", "_")
print('pv name: ', pvName)
print('ONTAP volume name: ', volumeName)

pv name:  pvc-db963a53-abf2-4ffa-9c07-8815ce78d506
ONTAP volume name:  trident_pvc_db963a53_abf2_4ffa_9c07_8815ce78d506


## Create snapshot

In [12]:
volume = Volume.find(name = volumeName)
timestamp = datetime.today().strftime("%Y%m%d_%H%M%S")
snapshot = Snapshot.from_dict({
    'name': 'jupyter_%s' % timestamp,
    'comment': 'Snapshot created from within a Jupyter Notebook',
    'volume': volume.to_dict()
})
response = snapshot.post()
print("API Response:")
print(response.http_response.text)

API Response:
{
  "uuid": "eeb78b35-684e-11ea-808d-d039ea06439f",
  "description": "POST /api/storage/volumes/691c30cc-4dac-11ea-808d-d039ea06439f/snapshots/?name=jupyter_20200317_180016",
  "state": "success",
  "message": "success",
  "code": 0,
  "start_time": "2020-03-17T12:58:00+00:00",
  "end_time": "2020-03-17T12:58:01+00:00",
  "_links": {
    "self": {
      "href": "/api/cluster/jobs/eeb78b35-684e-11ea-808d-d039ea06439f"
    }
  }
}


## Optional: Retrieve details for newly created snapshot

In [13]:
snapshot.get()
print(json.dumps(snapshot.to_dict(), indent=2))

{
  "create_time": "2020-03-17T12:58:01+00:00",
  "volume": {
    "name": "trident_pvc_db963a53_abf2_4ffa_9c07_8815ce78d506",
    "uuid": "691c30cc-4dac-11ea-808d-d039ea06439f",
    "_links": {
      "self": {
        "href": "/api/storage/volumes/691c30cc-4dac-11ea-808d-d039ea06439f"
      }
    }
  },
  "comment": "Snapshot created from within a Jupyter Notebook",
  "name": "jupyter_20200317_180016",
  "svm": {
    "name": "ai221_data",
    "uuid": "e6121682-3224-11ea-8196-d039ea06490a",
    "_links": {
      "self": {
        "href": "/api/svm/svms/e6121682-3224-11ea-8196-d039ea06490a"
      }
    }
  },
  "uuid": "e0ccde47-da43-433d-aa62-31aea7294daa",
  "_links": {
    "self": {
      "href": "/api/storage/volumes/691c30cc-4dac-11ea-808d-d039ea06439f/snapshots/e0ccde47-da43-433d-aa62-31aea7294daa"
    }
  }
}


## Optional: Retrieve a list of all snapshots that exist for the volume

In [7]:
totalSnapshots = 0
for volumeSnapshot in Snapshot.get_collection(volume.uuid, max_records = 256) :
    totalSnapshots += 1
    volumeSnapshot.get()
    print("Snapshot #%s:" % totalSnapshots)
    print(json.dumps(volumeSnapshot.to_dict(), indent=2), "\n")
    
print("Total Snapshots: %s" % totalSnapshots)

Snapshot #1:
{
  "create_time": "2020-02-12T15:42:51+00:00",
  "volume": {
    "name": "trident_pvc_db963a53_abf2_4ffa_9c07_8815ce78d506",
    "uuid": "691c30cc-4dac-11ea-808d-d039ea06439f",
    "_links": {
      "self": {
        "href": "/api/storage/volumes/691c30cc-4dac-11ea-808d-d039ea06439f"
      }
    }
  },
  "name": "clone_kfp_clone_202002.0",
  "svm": {
    "name": "ai221_data",
    "uuid": "e6121682-3224-11ea-8196-d039ea06490a",
    "_links": {
      "self": {
        "href": "/api/svm/svms/e6121682-3224-11ea-8196-d039ea06490a"
      }
    }
  },
  "uuid": "a2e40522-86bd-4b2d-91ac-fcfb2fedade3",
  "_links": {
    "self": {
      "href": "/api/storage/volumes/691c30cc-4dac-11ea-808d-d039ea06439f/snapshots/a2e40522-86bd-4b2d-91ac-fcfb2fedade3"
    }
  }
} 

Snapshot #2:
{
  "create_time": "2020-02-25T14:55:22+00:00",
  "volume": {
    "name": "trident_pvc_db963a53_abf2_4ffa_9c07_8815ce78d506",
    "uuid": "691c30cc-4dac-11ea-808d-d039ea06439f",
    "_links": {
      "self": {